In [142]:
!pip install pybaseball

In [143]:
import pandas as pd 
import numpy as np 
import requests
import io
from pybaseball import playerid_lookup, cache, statcast_pitcher, playerid_reverse_lookup

cache.enable()

In [144]:
###OPTIONS#####


PITCHER = "trevor bauer"
START = "2019-01-01"
END = "2020-12-01"
MIN_P = 100 
start_year = int(START[0:4])
end_year = int(END[0:4]) 
################

In [145]:
PITCHER = PITCHER.split()
p_first = PITCHER[0]
p_last = PITCHER[1]
p_id = playerid_lookup(p_last,p_first)

In [146]:
if p_id.empty: 
  print(p_first,p_last, "NOT FOUND")
  assert False
else: 
  p_id = p_id['key_mlbam'].iloc[0]

In [147]:

pitches = statcast_pitcher(START,END,p_id)
pitches.columns

Gathering Player Data


Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [148]:
pitches = pitches.sort_values(by=['game_date','at_bat_number','pitch_number'])

In [149]:
to_drop = ['release_pos_x','release_pos_y','release_pos_z','player_name','pitcher','spin_dir','spin_rate_deprecated',
          'break_angle_deprecated','break_length_deprecated','zone','des','game_type','p_throws','home_team','away_team',
           'hit_location','pfx_x','pfx_z', 'plate_x', 'plate_z','inning_topbot','hc_x','hc_y','tfs_deprecated', 
           'tfs_zulu_deprecated', 'fielder_2','sv_id','vx0', 'vy0', 'vz0', 'ax', 'ay', 'az','sz_top','sz_bot',
           'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'pitcher.1', 'fielder_2.1', 
           'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
            'home_score', 'away_score', 'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score', 'pitch_name',
            'estimated_ba_using_speedangle','umpire','release_extension','woba_value','woba_denom','bb_type','spin_axis','delta_home_win_exp','at_bat_number','events','description'
           ]  #zone may be interesting, further research requried 

pitches = pitches.drop(to_drop,axis=1)

In [150]:

pitches = pitches.replace("FT","SIFT")
pitches = pitches.replace("SI","SIFT")
pitches = pitches.replace("CU","CUKC")
pitches = pitches.replace("KC","CUKC")
pitches = pitches[pitches.pitch_type.notna()]
ptypes = list(pitches.pitch_type.unique())
pitches = pitches[pitches.pitch_type != 'PO']
pitches = pitches[pitches.pitch_type != 'IN']
ptypes

['FF', 'SIFT', 'CH', 'FC', 'SL', 'CUKC']

In [151]:
pitches[0:20]

,pitch_type,game_date,release_speed,batter,stand,type,balls,strikes,game_year,on_3b,on_2b,on_1b,outs_when_up,inning,release_spin_rate,game_pk,estimated_woba_using_speedangle,babip_value,iso_value,launch_speed_angle,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,delta_run_exp
4954,FF,2019-03-30,95.2,596146,L,B,0,0,2019,NaN,NaN,NaN,0,1,2321.0,566963,NaN,NaN,NaN,NaN,1,0,0,Standard,Standard,0.037
4953,FF,2019-03-30,94.7,596146,L,S,1,0,2019,NaN,NaN,NaN,0,1,2209.0,566963,NaN,NaN,NaN,NaN,2,0,0,Standard,Standard,-0.050
4952,FF,2019-03-30,95.0,596146,L,X,1,1,2019,NaN,NaN,NaN,0,1,2384.0,566963,0.000,0.0,0.0,3.0,3,0,0,Standard,Standard,-0.236
4951,FF,2019-03-30,93.7,593871,L,S,0,0,2019,NaN,NaN,NaN,1,1,2152.0,566963,NaN,NaN,NaN,NaN,1,0,0,Standard,Standard,-0.030
4950,SIFT,2019-03-30,94.6,593871,L,S,0,1,2019,NaN,NaN,NaN,1,1,2189.0,566963,NaN,NaN,NaN,NaN,2,0,0,Standard,Standard,-0.041
4949,CH,2019-03-30,89.4,593871,L,B,0,2,2019,NaN,NaN,NaN,1,1,1548.0,566963,NaN,NaN,NaN,NaN,3,0,0,Standard,Standard,0.019
4948,CH,2019-03-30,87.3,593871,L,S,1,2,2019,NaN,NaN,NaN,1,1,1212.0,566963,NaN,0.0,0.0,NaN,4,0,0,Standard,Standard,-0.125
4947,FF,2019-03-30,95.6,443558,R,S,0,0,2019,NaN,NaN,NaN,2,1,2243.0,566963,NaN,NaN,NaN,NaN,1,0,0,Standard,Standard,-0.019
4946,FC,2019-03-30,87.8,443558,R,B,0,1,2019,NaN,NaN,NaN,2,1,2629.0,566963,NaN,NaN,NaN,NaN,2,0,0,Standard,Strategic,0.012
4945,FC,2019-03-30,86.2,443558,R,S,1,1,2019,NaN,NaN,NaN,2,1,2490.0,566963,NaN,NaN,NaN,NaN,3,0,0,Standard,Strategic,-0.028


In [152]:
batter_versus = {}
cat_feats = ['stand','type','launch_speed_angle','if_fielding_alignment','of_fielding_alignment',
            'game_year']
            
def collapseEvents(df):
    evns = {'caught_stealing_2b' : 'out', 'double' : 'double', 'field_error': 'error', 'field_out': 'out', 'fielders_choice' : 'out', 'grounded_into_double_play' : 'out',
             'hit_by_pitch': 'hbp', 'home_run' : 'hr', 'sac_bunt': 'sac', 'sac_fly' : 'sac', 'single' : 'single', 'strikeout' : 'so', 'strikeout_double_play': 'so', 
              'walk' : 'walk'}
    des = {'ball' : 'ball', 'blocked_ball' : 'blocked_ball','bunt_foul_tip': 'mb','foul_bunt' : 'mb', 'called_strike':'cs','foul':'foul','hit_by_pitch':'hbp',
           'hit_into_play' : 'hip', 'missed_bunt' : 'mb', 'swinging_strike' : 'ss', 'swinging_strike_blocked' : 'ss' }
    df['events'] = df['events'].map(evns)
    df['description'] = df['description'].map(des)
    return df 

def splitGames(df): 
  df = df.groupby('game_pk')
  return [df.get_group(x) for x in df.groups ]

def gameOps(games):
  for game in games: 
    pc = {}
    #game['last'] = game.pitch_type.shift(1)
    #game['last2'] = game.pitch_type.shift(2)
    #game['last3'] = game.pitch_type.shift(3)
    #game['events'] = game.events.shift(1)
    #game['description'] = game.description.shift(1)
    #game['release_speed'] = game.release_speed.shift(1)
    #game['release_spin_rate'] = game.release_spin_rate.shift(1)
    #game['launch_speed_angle'] = game.launch_speed_angle.shift(1)
    #game['type'] = game['type'].shift(1)
    for pt in ptypes: 
      game[f'{pt}ct'] = 0 
      pc[pt] = 0 
    for i,row in game.iterrows():
      for pt in ptypes:
        game[f'{pt}ct'].loc[i] = pc[pt]
      pc[row.pitch_type] = pc[row.pitch_type] + 1 
      
  return pd.concat(games)

def statcast_batter_pitch_arsenal(ptype, year: int, minPA: int = 25) -> pd.DataFrame:
    url = f"https://baseballsavant.mlb.com/leaderboard/pitch-arsenal-stats?type=batter&pitchType={ptype}&year={year}&team=&min={minPA}&csv=true"
    res = requests.get(url, timeout=None).content
    data = pd.read_csv(io.StringIO(res.decode('utf-8')))
    return data

def getBatter(id,pt):
  rvp100 = 0 
  wpc = 0 
  hhp = 0 
  pitches = 0 
  for i in range(2014,end_year):
    
    data = statcast_batter_pitch_arsenal(pt,i)
    data = data[data.player_id.isin([id])]
    if data.empty:
      continue 
    
    pitches += data.pitches.iloc[0]
    rvp100 += data.run_value_per_100.iloc[0] * data.pitches.iloc[0]
    wpc += data.whiff_percent.iloc[0] * data.pitches.iloc[0]
    hhp += data.hard_hit_percent.iloc[0] * data.pitches.iloc[0]
  if pitches == 0: 
    return
  rvp100 /= pitches
  wpc /= pitches
  hhp /= pitches
  batter_versus[id][pt] = [rvp100,wpc,hhp]
def buildBatters():

  batters = list(pitches.batter.unique())

  for batter in batters: 
      batter_versus[batter] = {}
      for pt in ptypes: 
          getBatter(batter,pt)
  for pt in ptypes: 
    pitches[f"rvp100{pt}"] = pitches.apply(lambda row: None if pt not in batter_versus[row.batter] else batter_versus[row.batter][pt][0],axis=1)
    pitches[f"wpc{pt}"] = pitches.apply(lambda row: None if pt not in batter_versus[row.batter] else batter_versus[row.batter][pt][1],axis=1)
    pitches[f"hhp{pt}"] = pitches.apply(lambda row: None if pt not in batter_versus[row.batter] else batter_versus[row.batter][pt][2],axis=1)

#pitches = collapseEvents(pitches)
games = splitGames(pitches)
pitches = gameOps(games)
pitches = pd.get_dummies(pitches,columns=cat_feats)
buildBatters()
pitches[0:60]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,pitch_type,game_date,release_speed,batter,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,release_spin_rate,game_pk,estimated_woba_using_speedangle,babip_value,iso_value,pitch_number,bat_score,fld_score,delta_run_exp,FFct,SIFTct,CHct,FCct,SLct,CUKCct,stand_L,stand_R,type_B,type_S,type_X,launch_speed_angle_1.0,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_Standard,of_fielding_alignment_Strategic,game_year_2019,game_year_2020,rvp100FF,wpcFF,hhpFF,rvp100SIFT,wpcSIFT,hhpSIFT,rvp100CH,wpcCH,hhpCH,rvp100FC,wpcFC,hhpFC,rvp100SL,wpcSL,hhpSL,rvp100CUKC,wpcCUKC,hhpCUKC
1485,FF,2019-09-15,91.5,527038,0,0,NaN,NaN,NaN,0,1,2710.0,564904,NaN,NaN,NaN,1,0,0,0.037,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0.010188,11.492962,35.675603,-2.121679,7.181704,31.077569,-0.244086,17.702151,31.847312,5.500000,8.000000,26.900000,0.038840,21.741362,17.459269,1.856055,15.859862,26.977855
1484,FF,2019-09-15,92.0,527038,1,0,NaN,NaN,NaN,0,1,2689.0,564904,0.260,0.0,0.0,2,0,0,-0.285,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0.010188,11.492962,35.675603,-2.121679,7.181704,31.077569,-0.244086,17.702151,31.847312,5.500000,8.000000,26.900000,0.038840,21.741362,17.459269,1.856055,15.859862,26.977855
1483,FF,2019-09-15,90.2,606466,0,0,NaN,NaN,NaN,1,1,2684.0,564904,NaN,NaN,NaN,1,0,0,-0.030,2,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1482,FC,2019-09-15,86.4,606466,0,1,NaN,NaN,NaN,1,1,2905.0,564904,NaN,NaN,NaN,2,0,0,-0.041,3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1481,CUKC,2019-09-15,78.3,606466,0,2,NaN,NaN,NaN,1,1,2787.0,564904,NaN,NaN,NaN,3,0,0,0.000,3,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1480,FF,2019-09-15,93.1,606466,0,2,NaN,NaN,NaN,1,1,2615.0,564904,NaN,NaN,NaN,4,0,0,0.019,3,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1479,CH,2019-09-15,84.3,606466,1,2,NaN,NaN,NaN,1,1,2344.0,564904,NaN,NaN,NaN,5,0,0,0.031,4,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1478,FF,2019-09-15,93.1,606466,2,2,NaN,NaN,NaN,1,1,2719.0,564904,0.126,0.0,0.0,6,0,0,-0.156,4,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1477,FF,2019-09-15,92.9,500871,0,0,NaN,NaN,NaN,2,1,2732.0,564904,NaN,NaN,NaN,1,0,0,0.015,5,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.185364,26.211330,31.044614,1.546443,17.108095,37.948733,-1.764929,20.288708,22.029642,1.511656,25.969325,34.085890,1.949771,28.488101,31.394394,0.660617,24.153737,25.293120
1476,FF,2019-09-15,91.1,500871,1,0,NaN,NaN,NaN,2,1,2627.0,564904,NaN,NaN,NaN,2,0,0,-0.021,6,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.185364,26.211330,31.044614,1.546443,17.108095,37.948733,-1.764929,20.288708,22.029642,1.511656,25.969325,34.085890,1.949771,28.488101,31.394394,0.660617,24.153737,25.293120


In [153]:
pitches = pitches.replace("SIFT","FF")
pitches = pitches.replace("FC","FF")
pitches = pitches.replace("SL","BRK")
pitches = pitches.replace("CUKC","BRK")
pitches = pitches.replace("CH","OFF")
labels = pitches.pitch_type 
pitches['on_3b'] = pitches.apply(lambda row: 1 if (row.on_3b == row.on_3b) else 0,axis=1)
pitches['on_2b'] = pitches.apply(lambda row: 1 if (row.on_2b == row.on_2b) else 0,axis=1)
pitches['on_1b'] = pitches.apply(lambda row: 1 if (row.on_1b == row.on_1b) else 0,axis=1)
to_drop = ['game_date','batter','game_pk','estimated_woba_using_speedangle','babip_value','iso_value','pitch_type']


num_feats = ['release_speed','balls','strikes','outs_when_up','inning','release_spin_rate','pitch_number','bat_score',
             'fld_score', 'FFct',	'CUKCct', 'SLct',	'SIFTct',	'CHct',	'rvp100FF',	'wpcFF',	'hhpFF',	'rvp100CUKC',	'wpcCUKC',
             'hhpCUKC',	'rvp100SL',	'wpcSL',	'hhpSL',	'rvp100SIFT',	'wpcSIFT',	'hhpSIFT',	'rvp100CH',	'wpcCH',	'hhpCH']
pitches = pitches.drop(to_drop,axis=1)
pitches[0:60]

,release_speed,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,release_spin_rate,pitch_number,bat_score,fld_score,delta_run_exp,FFct,SIFTct,CHct,FCct,SLct,CUKCct,stand_L,stand_R,type_B,type_S,type_X,launch_speed_angle_1.0,launch_speed_angle_2.0,launch_speed_angle_3.0,launch_speed_angle_4.0,launch_speed_angle_5.0,launch_speed_angle_6.0,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_Standard,of_fielding_alignment_Strategic,game_year_2019,game_year_2020,rvp100FF,wpcFF,hhpFF,rvp100SIFT,wpcSIFT,hhpSIFT,rvp100CH,wpcCH,hhpCH,rvp100FC,wpcFC,hhpFC,rvp100SL,wpcSL,hhpSL,rvp100CUKC,wpcCUKC,hhpCUKC
1485,91.5,0,0,0,0,0,0,1,2710.0,1,0,0,0.037,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0.010188,11.492962,35.675603,-2.121679,7.181704,31.077569,-0.244086,17.702151,31.847312,5.500000,8.000000,26.900000,0.038840,21.741362,17.459269,1.856055,15.859862,26.977855
1484,92.0,1,0,0,0,0,0,1,2689.0,2,0,0,-0.285,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,0.010188,11.492962,35.675603,-2.121679,7.181704,31.077569,-0.244086,17.702151,31.847312,5.500000,8.000000,26.900000,0.038840,21.741362,17.459269,1.856055,15.859862,26.977855
1483,90.2,0,0,0,0,0,1,1,2684.0,1,0,0,-0.030,2,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1482,86.4,0,1,0,0,0,1,1,2905.0,2,0,0,-0.041,3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1481,78.3,0,2,0,0,0,1,1,2787.0,3,0,0,0.000,3,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1480,93.1,0,2,0,0,0,1,1,2615.0,4,0,0,0.019,3,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1479,84.3,1,2,0,0,0,1,1,2344.0,5,0,0,0.031,4,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1478,93.1,2,2,0,0,0,1,1,2719.0,6,0,0,-0.156,4,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0.586721,11.143054,34.636823,2.513296,6.435754,51.721899,-0.288124,31.132963,35.213762,0.794624,12.490323,34.800000,1.476860,23.361157,32.233058,0.469091,18.681322,40.512562
1477,92.9,0,0,0,0,0,2,1,2732.0,1,0,0,0.015,5,0,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.185364,26.211330,31.044614,1.546443,17.108095,37.948733,-1.764929,20.288708,22.029642,1.511656,25.969325,34.085890,1.949771,28.488101,31.394394,0.660617,24.153737,25.293120
1476,91.1,1,0,0,0,0,2,1,2627.0,2,0,0,-0.021,6,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0.185364,26.211330,31.044614,1.546443,17.108095,37.948733,-1.764929,20.288708,22.029642,1.511656,25.969325,34.085890,1.949771,28.488101,31.394394,0.660617,24.153737,25.293120


In [154]:
print(pitches.isnull().sum())
'''
pitches['release_speed'] = pitches['release_speed'].fillna(pitches.release_speed.mean())
pitches['events'] = pitches['events'].fillna("None")
pitches['description'] = pitches['events'].fillna("description")
pitches['type'] = pitches['type'].fillna("None") #maybe change
pitches['release_spin_rate'] = pitches['release_speed'].fillna(pitches.release_spin_rate.mean())
pitches['launch_speed_angle'] = pitches['launch_speed_angle'].fillna("None")
pitches['last'] = pitches['last'].fillna("None")
pitches['last2'] = pitches['last2'].fillna("None")
pitches['last3'] = pitches['last3'].fillna("None")
'''


release_speed                             0
balls                                     0
strikes                                   0
on_3b                                     0
on_2b                                     0
on_1b                                     0
outs_when_up                              0
inning                                    0
release_spin_rate                       186
pitch_number                              0
bat_score                                 0
fld_score                                 0
delta_run_exp                             0
FFct                                      0
SIFTct                                    0
CHct                                      0
FCct                                      0
SLct                                      0
CUKCct                                    0
stand_L                                   0
stand_R                                   0
type_B                                    0
type_S                          

'\npitches[\'release_speed\'] = pitches[\'release_speed\'].fillna(pitches.release_speed.mean())\npitches[\'events\'] = pitches[\'events\'].fillna("None")\npitches[\'description\'] = pitches[\'events\'].fillna("description")\npitches[\'type\'] = pitches[\'type\'].fillna("None") #maybe change\npitches[\'release_spin_rate\'] = pitches[\'release_speed\'].fillna(pitches.release_spin_rate.mean())\npitches[\'launch_speed_angle\'] = pitches[\'launch_speed_angle\'].fillna("None")\npitches[\'last\'] = pitches[\'last\'].fillna("None")\npitches[\'last2\'] = pitches[\'last2\'].fillna("None")\npitches[\'last3\'] = pitches[\'last3\'].fillna("None")\n'

In [155]:
print(len(cat_feats),len(num_feats),pitches.shape)

6 29 (4954, 55)


In [156]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


#for cat in cat_feats:
#pitches[cat] = pitches[cat].astype('str')
'''
pitches['events'] = pitches['events'].fillna("None")
pitches['description'] = pitches['events'].fillna("description")
pitches['type'] = pitches['type'].fillna("None") 
pitches['launch_speed_angle'] = pitches['launch_speed_angle'].fillna("None")
pitches['last'] = pitches['last'].fillna("None")
pitches['last2'] = pitches['last2'].fillna("None")
pitches['last3'] = pitches['last3'].fillna("None")
pitches['of_fielding_alignment'] = pitches['of_fielding_alignment'].fillna("None")
pitches['if_fielding_alignment'] = pitches['if_fielding_alignment'].fillna("None")
'''
num_pipe = Pipeline(
    [
     
     ('imp', SimpleImputer(missing_values=np.nan, strategy='mean')), 
     ('scaler', StandardScaler()),
     ]
     )

cat_pipe = Pipeline(
    [
      ('imp', SimpleImputer(missing_values=np.nan, strategy='constant',fill_value="none")),
      ('ohe',OneHotEncoder())
    ]
)

f_p = ColumnTransformer([
   ("num",num_pipe,num_feats)
  # ("cat",cat_pipe,cat_feats)
]
)
p_ready = f_p.fit_transform(pitches)

In [157]:
from sklearn import preprocessing
le = preprocessing.OneHotEncoder()
#print(labels)
labels_t = np.array(pd.get_dummies(labels))

In [167]:
seqs = []
seqlabs = []
for i in range(0,len(p_ready)-3):
  seqs.append(p_ready[i:i+3])
  seqlabs.append(labels_t[i+3])
seqs = np.asarray(seqs)
seqlabs = np.asarray(seqlabs)

In [168]:
seqs.shape

(4951, 3, 29)

In [170]:
from sklearn.model_selection import train_test_split
xtr, xte, ytr, yte = train_test_split(
  seqs, seqlabs, test_size=0.1, random_state=42)
ytr.shape

(4455, 3)

In [171]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam
clf = DecisionTreeClassifier(random_state=0)

'''
print(cross_val_score(clf, p_ready, labels_t, cv=10))

clf = svm.SVC()
print(cross_val_score(clf, p_ready, labels_t, cv=10))

clf = RandomForestClassifier(max_depth=15)
print(cross_val_score(clf, p_ready, labels_t, cv=10))

'''
'''
samples = [i for i in range (1,200)]
abc = np.array([[samples[i:i+3],samples[i:i+3]] for i in range (0,180)]).astype(np.float32).reshape(180,2,3)
defg = [samples[4 + i] for i in range(0,180)]
print(abc,defg)
abc /= 150
#abc = np.array([[1,2,3],[5,6,7],[8,9,10],[0,1,2],[12,13,14]])
#defg = np.array([[6],[8],[11],[5],[15]])
abc = np.asarray(abc)
defg = np.asarray(defg)
print(abc)
'''

model = Sequential()
#model.add(Embedding(10))
model.add(LSTM(40,return_sequences=True))
model.add(LSTM(40))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.1))
#model.add(Dense(100, input_dim=150, activation='relu'))
model.add(Dense(3,activation='softmax'))


  #model.add(Dense(3, activation='softmax'))
opt = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])




model.fit(xtr,ytr,epochs=200,validation_data=(xte,yte))



Epoch 1/200
140/140 [==============================] - 6s 12ms/step - loss: 1.0815 - accuracy: 0.4607 - val_loss: 1.0256 - val_accuracy: 0.5887
Epoch 2/200
140/140 [==============================] - 1s 6ms/step - loss: 0.9991 - accuracy: 0.6142 - val_loss: 0.9349 - val_accuracy: 0.5887
Epoch 3/200
140/140 [==============================] - 1s 6ms/step - loss: 0.9099 - accuracy: 0.6093 - val_loss: 0.8679 - val_accuracy: 0.5887
Epoch 4/200
140/140 [==============================] - 1s 6ms/step - loss: 0.8481 - accuracy: 0.6003 - val_loss: 0.8533 - val_accuracy: 0.5887
Epoch 5/200
140/140 [==============================] - 1s 6ms/step - loss: 0.8192 - accuracy: 0.6232 - val_loss: 0.8491 - val_accuracy: 0.5887
Epoch 6/200
140/140 [==============================] - 1s 6ms/step - loss: 0.8334 - accuracy: 0.5985 - val_loss: 0.8479 - val_accuracy: 0.5887
Epoch 7/200
140/140 [==============================] - 1s 6ms/step - loss: 0.8294 - accuracy: 0.6162 - val_loss: 0.8463 - val_accuracy: 0.588

KeyboardInterrupt: ignored